In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext rpy2.ipython

import ipyparallel as ipp
import matplotlib.pyplot as plt
import msprime
import numpy as np
import pandas as pd
import PIED
import toytree

from collections import Counter
from IPython.display import display, SVG

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)


In [11]:
tree = PIED.Core("stattest")
tre, _ = tree._simulate()
_ = tre.draw()

<svg class="toyplot-canvas-Canvas" height="360.0px" id="t5f1295933688498597b39d9a3300253e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 265.0 360.0" width="265.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 r5 r6 r7 r8 r9 r10 r11 r12 r13 r14 r15 r16 r17 r18 r19

# Implement the gamma statistic (Pybus & Harvey)

In [12]:
def gamma_stat(tre):
    N = tre.ntips
    # Get internode distances which will be held in gs
    dists = np.array(sorted([float(x) for x in tre.get_node_values(feature="height", show_root=True) if not x == ""], reverse=True))
    shift = np.append(dists[1:], [0])
    gs = dists - shift

    js = np.arange(2, N+1)
    T = np.sum(js * gs)

    cumsum = 0
    for i in range(2, N):
        for k in range(2, i+1):
            cumsum += k * gs[k-2]

    denom = T * np.sqrt(1/(12 * (N - 2)))

    stat = ((1/(N - 2)) * cumsum - (T/2))/denom

    return stat
gamma_stat(tre)    

0.16918375908265654

## Do we get the same result as ape?

In [13]:
newick = tre.write()

In [14]:
%%R -i newick
library(ape)
tree = read.tree(text=newick)
tree_gamma <- ape::gammaStat(tree)
print(tree_gamma)

[1] 0.1691968
